This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to rotation splines](index.ipynb)

# Kochanek--Bartels-like Rotation Splines

In [ ]:
import numpy as np

[helper.py](helper.py)

In [ ]:
from helper import angles2quat, animate_rotations, display_animation

[splines.quaternion.KochanekBartels](../python-module/splines.quaternion.rst#splines.quaternion.KochanekBartels)

In [ ]:
from splines.quaternion import KochanekBartels

In [ ]:
rotations = [
    angles2quat(0, 0, 0),
    angles2quat(90, 0, 0),
    angles2quat(90, 90, 0),
    angles2quat(90, 90, 90),
]

## Uniform Catmull--Rom

In [ ]:
s = KochanekBartels(rotations)

In [ ]:
times = np.linspace(s.grid[0], s.grid[-1], 100)

In [ ]:
ani = animate_rotations(s.evaluate(times), figsize=(4, 3))

In [ ]:
display_animation(ani, default_mode='reflect')

## Non-Uniform Catmull--Rom

In [ ]:
grid = 0, 0.2, 0.9, 1.2

In [ ]:
s = KochanekBartels(rotations, grid)

In [ ]:
times = np.linspace(s.grid[0], s.grid[-1], 100)

In [ ]:
ani = animate_rotations(s.evaluate(times), figsize=(4, 3))

In [ ]:
display_animation(ani, default_mode='reflect')

## TCB

In [ ]:
s = KochanekBartels(rotations, tcb=[0, 1, 1])

In [ ]:
times = np.linspace(s.grid[0], s.grid[-1], 100)

In [ ]:
ani = animate_rotations(s.evaluate(times), figsize=(4, 3))

In [ ]:
display_animation(ani, default_mode='reflect')

## Edge Cases

* 0 or 1 quaternions: not allowed
* 2 quaternions: Slerp
* 180° rotations (dot product = 0)?
* ...

2 quaternions:

In [ ]:
rotations = [
    angles2quat(0, 0, 0),
    angles2quat(90, 90, 90),
]

In [ ]:
s = KochanekBartels(rotations)

In [ ]:
times = np.linspace(s.grid[0], s.grid[-1], 100)

In [ ]:
ani = animate_rotations(s.evaluate(times), figsize=(4, 3))

In [ ]:
display_animation(ani, default_mode='reflect')